# Magnetic flowlines library

This is a little library for generating 3d-flowlines suitable for plotting in matplotlib or plot.ly. 

This assumes you are generating your vector field as the magnetic field induced by a current in a curve, surface or 3-dimensional solid. i.e. we are computing the flow lines of the vector field

$$\vec B(\vec p) = \int_C \frac{\vec I \times (\vec x - \vec p)}{|\vec x - \vec p|^3} dx$$

where $dx$ is the element of length, surface area or volume, depending on the type of object $C$ we are parametrizing. 

The routine takes a sympy algebraic expression for your curve, surface or solid.  You similarly provide a sympy description of your current field.  

The code partitions 3-dimensional space into a lattice, and performs a small integration of the vector field at the lattice points, returning the answer as a list of individual flow lines.  A single flowline will be a 3-element list (x,y,z coordinates) of lists.  This is plug-and-play with plot.ly and matplotlib. 

  * * * 
  
**flowLines** detailed description:

Call flowlines with two arguments: *flowLines(F,I)*

 * F is the 'parametrization packet', it is a list that provides the details of your parametric object.  $F = [f, D, I]$ with:
     * f is a sympy $3\times 1$ matrix expression, in 1, 2 or 3 variables. 
     * D is a list of the sympy variable names used in f. 
     * $I$ is the bounds of integration. If $D=[t_1,t_2]$ then we expect $I=[[a_1,b_1],[a_2,b_2]]$ where we take $t_1$ between $a_1$ and $b_1$, etc. 
 * I is the current expression.  This will be a sympy $n \times 1$ matrix expression where $n$ is the number of variables, i.e $n=len(D)$.  This gives the pull-back of the current vector field to the domain of $f$. 
 
  * * *

**Example**: Say we wish to describe a conducting infinite helix. 

$$ f(t) = (sin t, cos t, t) $$

and say the current is uniform, constant length equal to one.  Then our arguments would be

$$F = [sp.Matrix([sp.sin(t), sp.cos(t), t]), [t], [[-np.inf, np.inf]]]$$

and the current packet would be

$$I = sp.Matrix([1])$$

*Note*: you would have to define $t$ as a sympy symbol before defining the above. 

**Example 2**: Say we wish to describe a conducting unit sphere, current running along the lines of latitude. 

**Example 3**: A variant of 2 with current following some other lines...


In [ ]:
import sympy as sp
import numpy as np
from scipy import integrate as INT
from sympy.utilities.autowrap import ufuncify
import itertools as it
from ipywidgets import FloatProgress
from IPython.display import display

## Takes four arguments, 1) the parametrization packet F, which is a 3-element list.
##                       2) the current packet I which is a sympy matrix. 
##                       3) the lattice of coordinates to compute at, LAT. 
##                      this is a 3-element list of lists of x, y, z coordinates
##                      for the lattice. 
##                       4) flow is a 2-element list, flow=[fs,fn]
##                      with fs=flow step size, fn=number of steps
## returns a pair 1) the flowlines and 2) a callable version of F, to help with
##  plotting.
def flowLines(F, Ipb, LAT, flow):
    f = F[0]  ## sympy function, matrix valued. 
    k = len(F[1]) ## number of parameters to f. F[1] is the variable names.
    ## F[2] is the integration bounds. 
    
    print("Computing:  Distortion", end="", flush=True)
    Df = sp.zeros(3,k)
    for i in range(k):
        Df[:,i] = sp.diff(f, F[1][i])
    Cf = sp.sqrt( (Df.transpose()*Df).det() ).simplify()
    ## Cf is the length/area/volume distortion of f. 
    print(". ", end="", flush=True)
    
    print("Integrand", end="", flush=True)
    x,y,z = sp.symbols('x y z')
    P = sp.Matrix([x,y,z])
    ## we have to push I through Df to get the current vector field. 
    I = Df*Ipb
    den = sp.Matrix(f-P)
    ITG = I.cross(den)
    den = den.dot(den)**(3/2)
    ITG = (ITG/den)
    
    ## let's make an entire routine that does Euler's method callable? 
    ## this is possibly something to experiment with, the level of abstraction.
    ## we could also just make the integration command callable. 
    ITGc = [ufuncify(F[1]+[x,y,z], ITG[i,0]) for i in range(3)]
    print(". ", end="", flush=True)
 
    ## compile the curve as well, as we need to check distances
    C = [ufuncify(F[1], F[0][i,0]) for i in range(3)]
    
    ## loop running through the lattice [X,Y,Z]=LAT computing flowlines at those 
    ##  coordinates.
    retval = []
    fp = FloatProgress(min=0, max=100, description="Flowline integration progress");     
    display(fp); ## progrss indicator
    print("Integrating", end="", flush=True)

    count=0; totc = LAT[0].shape[0]*LAT[0].shape[1]*LAT[0].shape[2];
    for i,j,k in it.product(range(LAT[0].shape[0]), range(LAT[0].shape[1]), 
                            range(LAT[0].shape[2])):
        # TODO: let's throw in a basic check to see if this starting coordinate
        #  is too close to the curve. 
        errFlag = False
        flowline = [ [LAT[0][i,j,k]], [LAT[1][i,j,k]], [LAT[2][i,j,k]] ]
        for s in range(flow[1]):
            dp = [INT.nquad(ITGc[l], F[2], 
                            args=(flowline[0][-1], flowline[1][-1], flowline[2][-1]),
                            opts=[{'epsrel' : 0}, {'epsabs' : 1e-3}])[0] for l in range(3)]
            ## here is a primitive check to see if our lattice point is too close to
            ## the curve.  Should be replaced with something more sophisticated... later.
            ldp = np.sqrt(sum([crd**2 for crd in dp]))
                    
            if (ldp > 1e+8):
                errFlag = True
                break
            ## assemble our flowline
            for l in range(3):
                flowline[l].append(flowline[l][-1]+flow[0]*dp[l]/ldp)
        count+=1
        fp.value = int(100*count/totc)

        ## let's not build the flow line if ldp is too large. 
        if errFlag == False:
            retval.append(flowline)
    fp.close()
    print(".", flush=True)

    return retval, C

In [ ]:
## test run - current around an infinite conducting line
t = sp.Symbol('t')
f = sp.Matrix([ t, 0, 0])
flows, C = flowLines([f, [t], [[-np.inf,np.inf]]], sp.Matrix([1]), 
                  list(np.mgrid[-1.1:1.1:10j, -1.1:1.1:10j, -1.1:1.1:10j]), [0.04,10] )

In [ ]:
## And plot!

import plotly.offline as py
#import plotly.plotly as py
import plotly.graph_objs as go
py.init_notebook_mode() # run at the start of every notebook

T = np.arange(-1,1, 0.01)

tracelist = [go.Scatter3d( ## the knot
    x=C[0](T),    y=C[1](T),     z=C[2](T),
    mode='lines',
    line=go.Line(color='#2050FF', width=18)
    )]

tipsx = [P[0][-1] for P in flows]
tipsy = [P[1][-1] for P in flows]
tipsz = [P[2][-1] for P in flows]

tracelist.append(go.Scatter3d(
    x=tipsx, y=tipsy, z=tipsz, mode='markers',
    marker = dict(
        size=2, color = 'yellow') ) )

for P in flows:
    tracelist.append( go.Scatter3d(x=P[0], y=P[1], z=P[2], 
                                   mode='lines',
                                   line=go.Line(color='#FF4040', width=3)) )

data=go.Data(tracelist)

layout = go.Layout(
    title='Magnetic Field around a straight line',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
prefix = 'Magnetic Fields/'
py.plot(fig, filename='electricline.html')  #creates new page (large)
#py.iplot(fig) #puts inline

In [ ]:
## test run - current around an infinite conducting helix
t = sp.Symbol('t')
f = sp.Matrix([ sp.cos(t), sp.sin(t), t])
flows, C = flowLines([f, [t], [[-np.inf,np.inf]]], sp.Matrix([1]), 
                  list(np.mgrid[-1.5:1.5:8j, -1.5:1.5:8j, -2*np.pi:2*np.pi:16j]), [0.04,10] )

In [ ]:
## And plot!

#import plotly.offline as py
import plotly.plotly as py
import plotly.graph_objs as go
#py.init_notebook_mode() # run at the start of every notebook

T = np.arange(-2*np.pi,2*np.pi, 0.01)

tracelist = [go.Scatter3d( ## the knot
    x=C[0](T),    y=C[1](T),     z=C[2](T),
    mode='lines',
    line=go.Line(color='#2050FF', width=18)
    )]

tipsx = [P[0][-1] for P in flows]
tipsy = [P[1][-1] for P in flows]
tipsz = [P[2][-1] for P in flows]

tracelist.append(go.Scatter3d(
    x=tipsx, y=tipsy, z=tipsz, mode='markers',
    marker = dict(
        size=2, color = 'yellow') ) )

for P in flows:
    tracelist.append( go.Scatter3d(x=P[0], y=P[1], z=P[2], 
                                   mode='lines',
                                   line=go.Line(color='#FF4040', width=3)) )

data=go.Data(tracelist)

layout = go.Layout(
    title='Magnetic Field around a conducting helix',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
prefix = 'Magnetic Fields/'
filename=prefix+'electrichelix.html'
py.plot(fig, filename=filename)  #creates new page (large)
#py.iplot(fig) #puts inline